<a href="https://colab.research.google.com/github/meenuraji/S11FINAL/blob/master/A11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from torchsummary import summary
import torch
import torch.nn as nn
import torch.nn.functional as F
from eva4modeltrainer import ModelTrainer

class Net(nn.Module):
    """
    Base network that defines helper functions, summary and mapping to device
    """
    def conv2d(self, in_channels, out_channels, kernel_size=(3,3), dilation=1, groups=1, padding=1, bias=False, padding_mode="zeros"):
      return [nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, groups=groups, dilation=dilation, padding=padding, bias=bias, padding_mode=padding_mode)]

    def separable_conv2d(self, in_channels, out_channels, kernel_size=(3,3), dilation=1, padding=1, bias=False, padding_mode="zeros"):
      return [nn.Conv2d(in_channels=in_channels, out_channels=in_channels, kernel_size=kernel_size, groups=in_channels, dilation=dilation, padding=padding, bias=bias, padding_mode=padding_mode),
              nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=(1,1), bias=bias)]

    def activate(self, l, out_channels, bn=True, dropout=0, relu=True):
      if bn:
        l.append(nn.BatchNorm2d(out_channels))
      if dropout>0:
        l.append(nn.Dropout(dropout))
      if relu:
        l.append(nn.ReLU())

      return nn.Sequential(*l)

    def create_conv2d(self, in_channels, out_channels, kernel_size=(3,3), dilation=1, groups=1, padding=1, bias=False, bn=True, dropout=0, relu=True, padding_mode="zeros"):
      return self.activate(self.conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, groups=groups, dilation=dilation, padding=padding, bias=bias, padding_mode=padding_mode), out_channels, bn, dropout, relu)

    def create_depthwise_conv2d(self, in_channels, out_channels, kernel_size=(3,3), dilation=1, padding=1, bias=False, bn=True, dropout=0, relu=True, padding_mode="zeros"):
      return self.activate(self.separable_conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, dilation=dilation, padding=padding, bias=bias, padding_mode=padding_mode),
                 out_channels, bn, dropout, relu)

    def __init__(self, name="Model"):
        super(Net, self).__init__()
        self.trainer = None
        self.name = name

    def summary(self, input_size):
      summary(self, input_size=input_size)

    def gotrain(self, optimizer, train_loader, test_loader, epochs, statspath, scheduler=None, batch_scheduler=False, L1lambda=0):
      self.trainer = ModelTrainer(self, optimizer, train_loader, test_loader, statspath, scheduler, batch_scheduler, L1lambda)
      self.trainer.run(epochs)

    def stats(self):
      return self.trainer.stats if self.trainer else None
      
class Quiz11(Net):
    def __init__(self, name="Model", dropout_value=0):
      self.num_channels=64
      super(Quiz11, self).__init__(name)
      self.conv1=nn.Sequential(nn.Conv2d(3,64,3,padding=1,bias=False),nn.ReLU(),nn.BatchNorm2d(64))

    

      self.conv2=nn.Sequential(nn.Conv2d(64,128,3,padding=1,bias=False),nn.MaxPool2d(2),nn.ReLU(),nn.BatchNorm2d(128))

      self.Res1=nn.Sequential(nn.Conv2d(128,128,3,padding=1,bias=False),nn.ReLU(),nn.BatchNorm2d(128),nn.Conv2d(128,128,3,padding=1,bias=False),nn.ReLU(),nn.BatchNorm2d(128))

      

      self.conv3=nn.Sequential(nn.Conv2d(128,256,3,padding=1,bias=False),nn.MaxPool2d(2),nn.ReLU(),nn.BatchNorm2d(256))

      self.conv4=nn.Sequential(nn.Conv2d(256,512,3,padding=1,bias=False),nn.MaxPool2d(2),nn.ReLU(),nn.BatchNorm2d(512))

      self.Res2=nn.Sequential(nn.Conv2d(512,512,3,padding=1,bias=False),nn.ReLU(),nn.BatchNorm2d(512),nn.Conv2d(512,512,3,padding=1,bias=False),nn.ReLU(),nn.BatchNorm2d(512),)

      self.pool1=nn.MaxPool2d(4)

      self.conv5=nn.Conv2d(512,10,1)



    def forward(self, x):
      x1 = self.conv1(x)
      x2 = self.conv2(x1)
      r1 = self.Res1(x2)
      x3 = x2+r1
      
      x3 = self.conv3(x3)
      
      x4 = self.conv4(x3)
      r2 = self.Res2(x4)
      x5 = x4+r2
      x5 = self.pool1(x5)
      
      out = F.adaptive_avg_pool2d(x5, 1)
      out = self.conv5(out)
      out = out.view(-1, 10)
      return F.log_softmax(out, dim=-1)
